In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
# Load the trained model
model= load_model('model.h5')

# load the encoder and scaler
with open('OHE_geography.pkl','rb') as file:
    OHE_geography=pickle.load(file)
    
with open('LE_gender.pkl','rb') as file:
    LE_gender = pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [4]:
# example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


we have to change these inputs using those models

In [8]:
input_df['Gender'] = LE_gender.transform([input_df['Gender']])
input_df['Gender']

c:\Users\HP\OneDrive\Desktop\WEB D\Projects\ANN project\my_env\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


0    1
Name: Gender, dtype: int32

In [9]:
geo_encoder = OHE_geography.transform([[input_data['Geography']]])
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=OHE_geography.get_feature_names_out(['Geography']))
geo_encoder_df

c:\Users\HP\OneDrive\Desktop\WEB D\Projects\ANN project\my_env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df = pd.concat([input_df.drop(['Geography'],axis=1) , geo_encoder_df],axis=1)

In [12]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
input_df = scaler.transform(input_df)

predict

In [15]:
prediction = model.predict(input_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


In [16]:
prediction

array([[0.02311792]], dtype=float32)

In [17]:
if prediction[0][0] > 0.5:
    print("The Customer is likely to Churn")
else:
    print("The Customer is not likely to churn")

The Customer is not likely to churn
